In [1]:
import collections
import sys
import numpy
import numpy as np
#import project_tests as tests
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, SimpleRNN

Using TensorFlow backend.


In [2]:
from keras.layers import Merge
from keras.layers import Lambda
from keras import backend as K
import keras

In [3]:
import spacy

nlp = spacy.load('en_core_web_sm')
doc = nlp(u"Autonomous cars shift four insurance liability toward manufacturers SFD")
for token in doc:
    print(token.text, token.pos_, token.head.text, token.head.pos_,
          [child for child in token.children])
noun_key = {}
reverse_noun_key = {}
verb_key = {}
reverse_verb_key = {}
number_key = {}
reverse_number_key = {}

Autonomous ADJ cars NOUN []
cars NOUN shift VERB [Autonomous]
shift VERB shift VERB [cars, liability]
four NUM liability NOUN []
insurance NOUN liability NOUN []
liability NOUN shift VERB [four, insurance, toward]
toward ADP liability NOUN [SFD]
manufacturers NOUN SFD PROPN []
SFD PROPN toward ADP [manufacturers]


In [12]:
questions = open('fourth_grade_questions.txt','r')
labels = open('fourth_grade_labels.txt','r')
info_sentences = open('fourth_grade_info_sentences.txt','r')
fact_sentences = open('fourth_grade_fact_sentences.txt','r')
standard_relations = open('fourth_grade_standard_relations.txt','r')

questions_list = []
labels_list = []
info_list = []
relations_list = []
facts_list = []

for sentence in questions:
    sentence = sentence.lower()
    questions_list.append(sentence)
    
for sentence in labels:
    labels_list.append(sentence)
    
for sentence in info_sentences:
    sentence = sentence.lower()
    info_list.append(sentence)

for sentence in fact_sentences:
    sentence = sentence.lower()
    facts_list.append(sentence)
    
for sentence in standard_relations:
    relations_list.append(sentence)

In [13]:
print(len(questions_list[0]))
print(len(labels_list[0]))
print(len(info_list[0]))
print((facts_list[0]))
print(len(relations_list[0]))

134
16
227
customer service representatives is a compound word

347


In [14]:
# Iterate over sentences and use Spacy
word_to_index = {}
index_to_word = {}
 
pad_index = 999999
num_index = 0
adj_index = 100000
verb_index = 200000
noun_index = 300000
punct_index = 400000
det_index = 500000
propn_index = 500000
adv_index = 600000
conj_index = 700000
else_index = 800000
rela_index = 900000
#etc etc

new_facts_list =[]
new_questions_list = []
new_info_list = []
new_relations_list = []

pad_up_to = 290

for sentence in facts_list:
    substitute_sentence = [] #replace each word with index
    doc = nlp(sentence)
    for token in doc:
        if token.pos_== "NUM": # if word is a num
            if (token.text,token.pos_) not in word_to_index.keys():
                #Number index is special. It is equal to the number itself
                #We will convert all numbers written in numbers to numbers
                if token.text == "one":
                    num_index = 1
                elif token.text == "two":
                    num_index = 2
                elif token.text == "three":
                    num_index = 3
                elif token.text == "four":
                    num_index = 4
                elif token.text == "five":
                    num_index = 5
                elif token.text == "six":
                    num_index = 6
                elif token.text == "seven":
                    num_index = 7
                elif token.text == "eight":
                    num_index = 8
                elif token.text == "nine":
                    num_index = 9
                elif token.text == "ten":
                    num_index = 10
                #We will stop at 10 numbers. For numeric numbers, we will simply convert to int. We can only go upto 99999.
                else:
                    try:
                        num_index = int(token.text)
                    except:
                        num_index = 99999 #If number is not recognized, just make up an index.
                word_to_index[(token.text,token.pos_)]= num_index
                index_to_word[num_index] = (token.text, token.pos_)
                #convert sentence to indices
                substitute_sentence.append(num_index)
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])
        
        if token.pos_== "ADJ": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= adj_index
                index_to_word[adj_index] = (token.text, token.pos_)
                substitute_sentence.append(adj_index)
                adj_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "VERB": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= verb_index
                index_to_word[verb_index] = (token.text, token.pos_)
                substitute_sentence.append(verb_index)
                verb_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "NOUN": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= noun_index
                index_to_word[noun_index] = (token.text, token.pos_)
                substitute_sentence.append(noun_index)
                noun_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "DET": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= det_index
                index_to_word[det_index] = (token.text, token.pos_)
                substitute_sentence.append(det_index)
                det_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "PROPN": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= propn_index
                index_to_word[propn_index] = (token.text, token.pos_)
                substitute_sentence.append(propn_index)
                propn_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "PUNCT": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= punct_index
                index_to_word[punct_index] = (token.text, token.pos_)
                substitute_sentence.append(punct_index)
                punct_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "ADV": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= adv_index
                index_to_word[adv_index] = (token.text, token.pos_)
                substitute_sentence.append(adv_index)
                adv_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "CONJ": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= conj_index
                index_to_word[conj_index] = (token.text, token.pos_)
                substitute_sentence.append(conj_index)
                conj_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        else: #anything else, dump in else_index
            if (token.text,"else") not in word_to_index.keys():
                word_to_index[(token.text,"else")]= else_index
                index_to_word[else_index] = (token.text, "else")
                substitute_sentence.append(else_index)
                else_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,"else")])
    
    #pad each and then add to the facts_list
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
    new_facts_list.append(substitute_sentence)
    
for sentence in info_list:
    substitute_sentence = [] #replace each word with index
    doc = nlp(sentence)
    for token in doc:
        if token.pos_== "NUM": # if word is a num
            if (token.text,token.pos_) not in word_to_index.keys():
                #Number index is special. It is equal to the number itself
                #We will convert all numbers written in numbers to numbers
                if token.text == "one":
                    num_index = 1
                elif token.text == "two":
                    num_index = 2
                elif token.text == "three":
                    num_index = 3
                elif token.text == "four":
                    num_index = 4
                elif token.text == "five":
                    num_index = 5
                elif token.text == "six":
                    num_index = 6
                elif token.text == "seven":
                    num_index = 7
                elif token.text == "eight":
                    num_index = 8
                elif token.text == "nine":
                    num_index = 9
                elif token.text == "ten":
                    num_index = 10
                #We will stop at 10 numbers. For numeric numbers, we will simply convert to int. We can only go upto 99999.
                else:
                    try:
                        num_index = int(token.text)
                    except:
                        num_index = 99999 #If number is not recognized, just make up an index.
                word_to_index[(token.text,token.pos_)]= num_index
                index_to_word[num_index] = (token.text, token.pos_)
                #convert sentence to indices
                substitute_sentence.append(num_index)
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])
        
        if token.pos_== "ADJ": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= adj_index
                index_to_word[adj_index] = (token.text, token.pos_)
                substitute_sentence.append(adj_index)
                adj_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "VERB": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= verb_index
                index_to_word[verb_index] = (token.text, token.pos_)
                substitute_sentence.append(verb_index)
                verb_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "NOUN": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= noun_index
                index_to_word[noun_index] = (token.text, token.pos_)
                substitute_sentence.append(noun_index)
                noun_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "DET": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= det_index
                index_to_word[det_index] = (token.text, token.pos_)
                substitute_sentence.append(det_index)
                det_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "PROPN": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= propn_index
                index_to_word[propn_index] = (token.text, token.pos_)
                substitute_sentence.append(propn_index)
                propn_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "PUNCT": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= punct_index
                index_to_word[punct_index] = (token.text, token.pos_)
                substitute_sentence.append(punct_index)
                punct_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "ADV": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= adv_index
                index_to_word[adv_index] = (token.text, token.pos_)
                substitute_sentence.append(adv_index)
                adv_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "CONJ": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= conj_index
                index_to_word[conj_index] = (token.text, token.pos_)
                substitute_sentence.append(conj_index)
                conj_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        else: #anything else, dump in else_index
            if (token.text,"else") not in word_to_index.keys():
                word_to_index[(token.text,"else")]= else_index
                index_to_word[else_index] = (token.text, "else")
                substitute_sentence.append(else_index)
                else_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,"else")])

    #pad each and then add to the facts_list
    for i in range(len(substitute_sentence),pad_up_to): 
        substitute_sentence.append(pad_index)
    new_info_list.append(substitute_sentence)
    
for sentence in questions_list:
    substitute_sentence = [] #replace each word with index
    doc = nlp(sentence)
    for token in doc:
        if token.pos_== "NUM": # if word is a num
            if (token.text,token.pos_) not in word_to_index.keys():
                #Number index is special. It is equal to the number itself
                #We will convert all numbers written in numbers to numbers
                if token.text == "one":
                    num_index = 1
                elif token.text == "two":
                    num_index = 2
                elif token.text == "three":
                    num_index = 3
                elif token.text == "four":
                    num_index = 4
                elif token.text == "five":
                    num_index = 5
                elif token.text == "six":
                    num_index = 6
                elif token.text == "seven":
                    num_index = 7
                elif token.text == "eight":
                    num_index = 8
                elif token.text == "nine":
                    num_index = 9
                elif token.text == "ten":
                    num_index = 10
                #We will stop at 10 numbers. For numeric numbers, we will simply convert to int. We can only go upto 99999.
                else:
                    try:
                        num_index = int(token.text)
                    except:
                        num_index = 99999 #If number is not recognized, just make up an index.
                word_to_index[(token.text,token.pos_)]= num_index
                index_to_word[num_index] = (token.text, token.pos_)
                #convert sentence to indices
                substitute_sentence.append(num_index)
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])
        
        if token.pos_== "ADJ": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= adj_index
                index_to_word[adj_index] = (token.text, token.pos_)
                substitute_sentence.append(adj_index)
                adj_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "VERB": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= verb_index
                index_to_word[verb_index] = (token.text, token.pos_)
                substitute_sentence.append(verb_index)
                verb_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "NOUN": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= noun_index
                index_to_word[noun_index] = (token.text, token.pos_)
                substitute_sentence.append(noun_index)
                noun_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "DET": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= det_index
                index_to_word[det_index] = (token.text, token.pos_)
                substitute_sentence.append(det_index)
                det_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "PROPN": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= propn_index
                index_to_word[propn_index] = (token.text, token.pos_)
                substitute_sentence.append(propn_index)
                propn_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "PUNCT": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= punct_index
                index_to_word[punct_index] = (token.text, token.pos_)
                substitute_sentence.append(punct_index)
                punct_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "ADV": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= adv_index
                index_to_word[adv_index] = (token.text, token.pos_)
                substitute_sentence.append(adv_index)
                adv_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        if token.pos_== "CONJ": # if word is a verb
            if (token.text,token.pos_) not in word_to_index.keys():
                word_to_index[(token.text,token.pos_)]= conj_index
                index_to_word[conj_index] = (token.text, token.pos_)
                substitute_sentence.append(conj_index)
                conj_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,token.pos_)])

        else: #anything else, dump in else_index
            if (token.text,"else") not in word_to_index.keys():
                word_to_index[(token.text,"else")]= else_index
                index_to_word[else_index] = (token.text, "else")
                substitute_sentence.append(else_index)
                else_index += 1
            else: #if word already in the dictionary, we still add the index to the substitute sentence
                substitute_sentence.append(word_to_index[(token.text,"else")])

    #pad each and then add to the facts_list
    for i in range(len(substitute_sentence),pad_up_to):
        substitute_sentence.append(pad_index)
    new_questions_list.append(substitute_sentence)
    

#For relations, we will use the hand-typed file for this example. For future studies, a function can easily be coded
#Since we hand-wrote the file, we made it easy to process by including tuple. Now we can use the word_to_index
for sentence in relations_list:
    substitute_sentence = []
    sent = sentence.split()
    for char in sent: #char could be a word or group of characters
        if (char, "NOUN") in word_to_index.keys():
            substitute_sentence.append(word_to_index[(char,"NOUN")])
        if (char, "VERB") in word_to_index.keys():
            substitute_sentence.append(word_to_index[(char,"VERB")])
        if (char, "ADJ") in word_to_index.keys():
            substitute_sentence.append(word_to_index[(char,"ADJ")])
        if (char, "rela") not in word_to_index.keys():
            word_to_index[(char, "rela")] = rela_index
            substitute_sentence.append(rela_index)
            rela_index += 1
        else: 
            substitute_sentence.append(word_to_index[(char,"rela")])
    
    for i in range(len(substitute_sentence),pad_up_to): 
        substitute_sentence.append(pad_index)
    new_relations_list.append(substitute_sentence)

word_to_index[("<PAD>","pad")] = 999999

In [16]:
print(len(new_facts_list[0]))
print(len(new_questions_list[0]))
print(len(new_info_list[0]))
print(len(new_relations_list[33]))

290
290
290
290


In [28]:
def vectorize_sentences(sentence_list):
    new_list = []
    for sentence in sentence_list:
        vectorized_sentence = []
        for index in sentence:
            vector = []
            for dgit in str(index):
                vector.append(int(dgit))
            vectorozeros = [0]*(6-len(vector))
            vectorozeros.extend(vector)
            vector = vectorozeros
            vectorized_sentence.append(vector)
        new_list.append(vectorized_sentence)
    return new_list
#check

vectorized_fact_list = np.asarray(vectorize_sentences(new_facts_list))
vectorized_questions = np.asarray(vectorize_sentences(new_questions_list)) 
vectorized_info = np.asarray(vectorize_sentences(new_info_list))
vectorized_relations = np.asarray(vectorize_sentences(new_relations_list))

In [29]:
print(vectorized_fact_list.shape)
print(vectorized_questions.shape)
print(vectorized_info.shape)
print(vectorized_relations.shape)

(34, 290, 6)
(34, 290, 6)
(34, 290, 6)
(34, 290, 6)


In [46]:
label_words_to_index = {}
label_index_to_words = {}
label_index = 0
label_words_to_index["\t"]=label_index
label_index_to_words[label_index]="\t"
label_index += 1
label_words_to_index["<PAD>"]=label_index
label_index_to_words[label_index]="<PAD>"
new_label_list = []
label_pad_up_to = 50
for sentence in labels_list:
    substitute_sentence = []
    sent = sentence.split()
    for char in sent: #char could be a word or group of characters
        if char in label_words_to_index.keys():
            substitute_sentence.append(label_words_to_index[char])
        else:
            label_index += 1
            label_words_to_index[char]=label_index
            label_index_to_words[label_index]=char
            substitute_sentence.append(label_words_to_index[char])
    for i in range(len(substitute_sentence),label_pad_up_to): #pad to regular
        substitute_sentence.append(label_words_to_index["<PAD>"])
        
    new_label_list.append(substitute_sentence)
    
#vectorize labels
vectorized_label_list = []
for sentence in new_label_list:
    sentence_vector = []
    for word in sentence:
        vector = [0]*len(label_words_to_index)
        vector[word] = 1
        sentence_vector.append(vector)
    vectorized_label_list.append(sentence_vector)
    
label_array = np.asarray(vectorized_label_list)

In [47]:
print(label_array.shape)
print(len(label_words_to_index))
print(len(label_index_to_words))

(34, 50, 48)
48
48


In [48]:
def logits_to_text(logits, dictionary):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """    
    return ' '.join([label_index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [49]:
def cascaded_reasoning(info_shape, questions_shape, facts_shape \
                       , relations_shape, labels_index_len):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    
    # TODO: Implement
    learning_rate = 0.01
    hidden_units = 128
    latent_dim = 128 
    # TODO: Build the layers
    
    LSTM_info_inputs = Input(shape=(None, info_shape[2]))
    LSTM_info = LSTM(hidden_units,return_state=True)
    LSTM_info_output, info_h, info_c=LSTM_info(LSTM_info_inputs)
    info_states = [info_h,info_c]
    
    LSTM_questions_inputs = Input(shape=(None, questions_shape[2]))
    LSTM_questions = LSTM(hidden_units,return_state=True)
    LSTM_questions_output, questions_h, questions_c=LSTM_questions(LSTM_questions_inputs)
    questions_states= [questions_h,questions_c]
    
    LSTM_facts_inputs = Input(shape=(None, facts_shape[2]))
    LSTM_facts = LSTM(hidden_units,return_state=True)
    LSTM_facts_output, facts_h, facts_c=LSTM_facts(LSTM_facts_inputs)
    facts_states= [facts_h,facts_c]
    
    LSTM_rela_inputs = Input(shape=(None, relations_shape[2]))
    LSTM_rela = LSTM(hidden_units,return_state=True)
    LSTM_rela_output,rela_h,rela_c=LSTM_rela(LSTM_rela_inputs)
    rela_states= [rela_h,rela_c]
    
    added_h = keras.layers.Add()([info_h,questions_h,facts_h,rela_h])
    added_c = keras.layers.Add()([info_c,questions_c,facts_c,rela_c])
    states = [added_h,added_c]
    
    decoder_inputs = Input(shape=(1, labels_index_len))
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_dense = Dense(labels_index_len, activation='softmax')

    all_outputs = []
    inputs = decoder_inputs
    for _ in range(label_pad_up_to):
        # Run the decoder on one timestep
        outputs, state_h, state_c = decoder_lstm(inputs,
                                             initial_state=states)
        outputs = decoder_dense(outputs)
        # Store the current prediction (we will concatenate all predictions later)
        all_outputs.append(outputs)
        # Reinject the outputs as inputs for the next loop iteration
        # as well as update the states
        inputs = outputs
        states = [state_h, state_c]

    # Concatenate all predictions
    decoder_outputs = Lambda(lambda x: K.concatenate(x, axis=1))(all_outputs)

    # Define and compile model as previously
    model_T = Model([LSTM_info_inputs, LSTM_questions_inputs, LSTM_facts_inputs, \
                     LSTM_rela_inputs, decoder_inputs], decoder_outputs)

    model_T.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model_T

#Example inputs: X1 = "There are four apples at the top of the tree." X2 = "What do you need to get the apples?"
# X3 = "Trees are high."  Label result = "Trees -> High"
#Example inputs: X1 = "There are four apples at the top of the tree." X2 = "What do you need to get the apples?"
# X3 = "Pick apples from tree."  Label result = "Apples -> On tree"
#Example inputs: X1 = "Apples->On tree, Tree->High" X2 = "What do you need to get the apples?"
# X3 = "You can go on tree with a ladder."  Label result = "Ladder"

In [50]:
#tests.test_model_final(model_final)
batch_size = 34

print(len(label_index_to_words))

cascaded_reasoning= cascaded_reasoning(vectorized_info.shape, vectorized_questions.shape, \
                       vectorized_fact_list.shape, vectorized_relations.shape, len(label_index_to_words))

print('cascaded_reasoning loaded')

# Prepare decoder input data that just contains the start character
# Note that we could have made it a constant hard-coded in the model
decoder_input_data = np.zeros((batch_size, 1, len(label_index_to_words)))
decoder_input_data[:, 0, label_words_to_index['\t']] = 1.

cascaded_reasoning.fit([vectorized_info, vectorized_questions, \
                        vectorized_fact_list, vectorized_relations, decoder_input_data], \
                       label_array, batch_size=batch_size, epochs=10, validation_split=0.1)

print("model fit")



48
cascaded_reasoning loaded
Train on 30 samples, validate on 4 samples
Epoch 1/10
30/30 [==============================] - 551s 18s/step - loss: 3.8827 - acc: 0.0033 - val_loss: 3.5070 - val_acc: 0.7800
Epoch 2/10
30/30 [==============================] - 4s 126ms/step - loss: 3.2308 - acc: 0.9080 - val_loss: 2.9829 - val_acc: 0.7800
Epoch 3/10
30/30 [==============================] - 3s 95ms/step - loss: 2.5316 - acc: 0.9080 - val_loss: 1.8108 - val_acc: 0.7800
Epoch 4/10
30/30 [==============================] - 3s 92ms/step - loss: 1.2660 - acc: 0.9080 - val_loss: 1.2264 - val_acc: 0.7800
Epoch 5/10
30/30 [==============================] - 3s 92ms/step - loss: 0.5790 - acc: 0.9080 - val_loss: 1.1702 - val_acc: 0.7800
Epoch 6/10
30/30 [==============================] - 3s 92ms/step - loss: 0.5011 - acc: 0.9080 - val_loss: 1.1783 - val_acc: 0.7800
Epoch 7/10
30/30 [==============================] - 3s 95ms/step - loss: 0.4686 - acc: 0.9080 - val_loss: 1.1581 - val_acc: 0.7800
Epoch 8/1

In [100]:
print(logits_to_text(cascaded_reasoning.predict([[vectorized_info[:1], vectorized_questions[:1]], \
                        [vectorized_fact_list[:1], vectorized_relations[:1]]], label_index_to_words)))

AttributeError: 'list' object has no attribute 'ndim'

In [ ]:
# TODO: Train the final model

#Example test:There are 4 doctors working in a clinic. Each doctor has 2 nurses assisting them. There
#are two receptionists, Jay and Molly, working at the reception.
#On Monday, 23 patients made appointments with each doctor. However, 6 of the
#patients did not show up. How many patients visited the clinic on Monday?

nlp = spacy.load('en_core_web_sm')
sentence = "There are 4 doctors working in a clinic. There are 4 doctors working in a clinic. \
          Each doctor has 2 nurses assisting them. There are two receptionists, Jay and Molly, \
          working at the reception. On Monday, 23 patients made appointments with each doctor. \
          However, 6 of the patients did not show up."
question = "How many patients visited the clinic on Monday?"
doc = nlp(sentence)
indx_n = 0
indx_v = 0
indx_num = 0
for token in doc:
    if token.pos_ == "NOUN":
        indx_n += 1
        relations.append("N%d = %s".format(indx_n, token.text))
        noun_key[token.text] = "N%d".format(indx_n)
        reverse_noun_key["N%d".format(indx_n)]=token.text
    if token.pos_ == "VERB":
        indx_v += 1
        relations.append("V%d = %s".format(indx_v, token.text))
        verb_key[token.text] = "V%d".format(indx_v)
        reverse_verb_key["V%d".format(indx_v)]=token.text
    if token.dep_ == "num" or token.dep_ == "num": #this is deprecated
        indx_num += 1
        relations.append("NUM%d = %s".format(indx_num, token.text))
        number_key[token.text] = "NUM%d".format(indx_num)
        reverse_number_key["NUM%d".format(indx_num)]=token.text

#We will not convert the numbers. Instead, we will use vector representations of the numbers. The machine can figure 
#out the relation between the numbers in the sentence. We will represent every word with a 10-digit vector. The 
#first digits will indicate the type of word; for example: representative can be represented by the vector 9000000602,  
#where 9 is for noun. Another example: work = 6000000052, where 6 is for verb. Now, for numbers, there will be zeros in the 
#beginning. For example, 3 will simply be 0000000003, and 24 = 0000000024. We will let the network figure out that these 
#represent actual numbers.   

#create variables to be used in assignments:
var_list = []
for indx_var in range(10):
    var_list[indx_var+1] = "VAR%d".format(indx_var+1) #Add 1 so that VAR list starts with VAR1
    
fact_list[0] = "A physician, medical practitioner, medical doctor, or simply doctor is a professional who practises medicine, \
which is concerned with promoting, maintaining, or restoring health through the study, diagnosis, and treatment \
of disease, injury, and other physical and mental impairments." 

#Now collect info about each of the nouns based on the question:
for noun in noun_list:
    #example: noun being focussed = doctors 
    for fact in fact_list:  
        inputs = [sentence, question, relations, facts]
        #we expect an output that will yield a sort of relation between the words that will be useful to answer the question:
        #these relations will be initially input by humans to train the network on what to look for
        #in this case, the output could be a list of four or five relations, each relation composed of up to four words: 
        #[doctor->practices->medicine, medicine->about->health, 23 patients->made->appointments, 4->doctors]
        #The output from this iteration will be ammended to the relations list and fed into the next iteration.
        #It is possible to have the network find the relations itself by stacking several of these networks. However, the
        #resulting mega-network will be too huge to train. A supercomputer may be necessary.   
        #For now, we will fix the question and info, and iterate over the fact sentences. Labels will correspond to facts. 